# Twitter Test

## Tweepy

How to get up-to-date user tweets?

In [1]:
# proxy
from fp.fp import FreeProxy  # https://github.com/jundymek/free-proxy

def gen_proxy_list(timeout=5, google_enable=False, anonym=False, filtered=False, https=False):
    return FreeProxy(
        timeout=timeout, 
        google=google_enable, 
        anonym=anonym,
        elite=filtered,
        https=https,
        rand=True).get_proxy_list(repeat=True)

http_proxies = gen_proxy_list(timeout=5)

In [2]:
http_proxies

['77.242.98.39:8080',
 '3.99.167.1:3128',
 '85.215.64.49:80',
 '34.81.72.31:80',
 '23.247.137.142:80',
 '103.152.112.120:80',
 '43.224.248.29:8085',
 '23.247.136.245:80',
 '23.247.136.254:80',
 '103.152.112.157:80',
 '103.152.112.159:80',
 '103.152.112.195:80',
 '103.152.112.186:80',
 '123.30.154.171:7777',
 '89.116.27.24:8888',
 '71.14.218.2:8080',
 '38.150.15.11:80',
 '47.88.59.79:82',
 '222.252.194.204:8080',
 '47.56.110.204:8989',
 '162.223.90.130:80',
 '133.18.234.13:80',
 '87.248.129.26:80',
 '66.29.154.103:3128',
 '190.103.177.131:80',
 '154.205.152.96:4145',
 '32.223.6.94:80',
 '13.38.153.36:80',
 '13.36.104.85:80',
 '13.36.87.105:3128',
 '13.37.73.214:80',
 '44.218.183.55:80',
 '44.195.247.145:80',
 '50.207.199.80:80',
 '50.207.199.83:80',
 '189.22.234.33:80',
 '50.207.199.82:80',
 '50.122.86.118:80',
 '189.22.234.40:80',
 '37.187.25.85:80',
 '54.67.125.45:3128',
 '184.169.154.119:80',
 '3.136.29.104:80',
 '13.208.56.180:80',
 '3.37.125.76:3128',
 '3.127.62.252:80',
 '18.228.1

In [3]:
from tweeterpy import TweeterPy

try:
    twitter = TweeterPy(proxies = {'http': '77.242.98.39:8080'})
except Exception as e:
    if e == 'invalid response':
        print(f"Caught a generic exception: {e}")

/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'twitter.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/jiezi/minicon

2025-02-21 17:51:08,383 [WARNING] :: Skipping API Updates. Couldn't get the latest API data.
2025-02-21 17:51:08,385 [INFO] :: API Updated Successfully.


In [4]:
uid = twitter.get_user_id('omarsar0')

/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [5]:
uid

'3448284313'

In [6]:
user_data = twitter.get_user_data('omarsar0')

/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [13]:
user_data['legacy']

{'created_at': 'Fri Sep 04 12:59:26 +0000 2015',
 'default_profile': False,
 'default_profile_image': False,
 'description': 'Building with AI agents @dair_ai • Prev: Meta AI, Galactica LLM, Elastic, PaperswithCode, PhD • I also teach how to leverage and build with LLMs & AI Agents ⬇️',
 'entities': {'description': {'urls': []},
  'url': {'urls': [{'display_url': 'dair-ai.thinkific.com',
     'expanded_url': 'https://dair-ai.thinkific.com/',
     'url': 'https://t.co/JBU5beHQNs',
     'indices': [0, 23]}]}},
 'fast_followers_count': 0,
 'favourites_count': 29023,
 'followers_count': 230904,
 'friends_count': 571,
 'has_custom_timelines': True,
 'is_translator': False,
 'listed_count': 3886,
 'location': '',
 'media_count': 2948,
 'name': 'elvis',
 'normal_followers_count': 230904,
 'pinned_tweet_ids_str': ['1892621241674301761'],
 'possibly_sensitive': False,
 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/3448284313/1565974901',
 'profile_image_url_https': 'https://pbs.t

In [60]:
tweet.get('api_rate_limit')

{'total_limit': 50,
 'remaining_requests_count': 47,
 'resets_after': '0 Hours, 14 Minutes, 33.63 Seconds',
 'reset_after_datetime_object': datetime.timedelta(seconds=873, microseconds=633757),
 'rate_limit_exhausted': False}

In [59]:
tweet = twitter.get_tweet(tweet_id='1892223536493703501')

/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [61]:
from tweeterpy import TweeterPy

try:
    twitter2 = TweeterPy(proxies = {'http': '85.215.64.49:80'})
except Exception as e:
    if e == 'invalid response':
        print(f"Caught a generic exception: {e}")

/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'twitter.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/jiezi/minicon

2025-02-21 18:37:50,507 [WARNING] :: Skipping API Updates. Couldn't get the latest API data.
2025-02-21 18:37:50,511 [INFO] :: API Updated Successfully.


In [62]:
tweet2 = twitter2.get_tweet(tweet_id='1892223536493703501')

/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [63]:
tweet2.get('api_rate_limit')

{'total_limit': 50,
 'remaining_requests_count': 49,
 'resets_after': '0 Hours, 14 Minutes, 59.08 Seconds',
 'reset_after_datetime_object': datetime.timedelta(seconds=899, microseconds=83628),
 'rate_limit_exhausted': False}

In [51]:
y = twitter.get_user_data('elonmask')

/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
future_datetime = datetime.datetime.fromtimestamp(current_time_unix_int) + dct_info['reset_after_datetime_object']

# Convert the future datetime back to a Unix timestamp (integer).
future_time_unix_int = int(future_datetime.timestamp())

In [64]:
import time
import datetime
print(int(time.time()), int(datetime.datetime.now().timestamp()))

1740135076 1740135076


In [ ]:
print(twitter.get_user_id('elonmask'))

In [ ]:
test_uid = twitter.get_user_id('elonmask')

In [ ]:
test_uid

In [ ]:
user_tweets = twitter.get_user_tweets("elonmusk",total=50)

In [ ]:
user_tweets.keys()

In [ ]:
user_tweets['has_next_page']

In [ ]:
len(user_tweets['data'])

In [ ]:

user_id = '28553'
query_endpoint = Path.USER_TWEETS_ENDPOINT
variables = {"userId": user_id, "count": 100, "includePromotedContent": True,
                "withQuickPromoteEligibilityTweetFields": True, "withVoice": True, "withV2Timeline": True}

request_payload = self._generate_request_data(
    query_endpoint, variables, additional_features=True)
data_path = ('data', 'user', 'result', 'timeline_v2',
                'timeline', 'instructions')
results = self._handle_pagination(**request_payload, end_cursor=end_cursor, data_path=data_path, total=total, pagination=pagination)

In [ ]:
import time, random
from tweeterpy import TweeterPy
from tweeterpy.util import RateLimitError

twitter = TweeterPy()
# login if required

user_tweets = []
has_more = True
cursor = None
while has_more:
    try:
        response = None
        response = twitter.get_user_tweets('elonmusk', end_cursor=cursor, pagination=False)
        user_tweets.extend(response['data'])
        has_more = response.get('has_next_page')
        api_rate_limits = response.get('api_rate_limit')
        limit_exhausted = api_rate_limits.get('rate_limit_exhausted')
        if has_more:
            cursor = response.get('cursor_endpoint')
        ## YOUR CUSTOM CODE HERE (DATA HANDLING, REQUEST DELAYS, SESSION SHUFFLING ETC.)
        ## time.sleep(random.uniform(7,10))
        if limit_exhausted:
            raise RateLimitError
    except Exception as error:
        print(error)
        break

In [ ]:
    def _generate_request_data(self, endpoint, variables=None, **kwargs):
        # fmt: off - Turns off formatting for this block of code. Just for the readability purpose.
        url = util.generate_url(domain=Path.API_URL, url_path=endpoint)
        query_params = {}
        if variables:
            query_params["variables"] = json.dumps(variables)
        if kwargs:
            features = FeatureSwitch().get_query_features(endpoint) or util.generate_features(**kwargs)
            query_params["features"] = json.dumps(features)
        # fmt: on   
        request_payload = {"url": url, "params": query_params}
        logger.debug(f"Request Payload => {request_payload}")
        return request_payload
        
        

    def get_user_tweets(self, user_id, with_replies=False, end_cursor=None, total=None, pagination=True):
        """Get Tweets from a user's profile.

        Args:
            user_id (int): User ID.
            with_replies (bool, optional): Set to True if want to get the tweets user replied to, from user's profile page. Defaults to False.
            end_cursor (str, optional): Last endcursor point. (To start from where you left off last time). Defaults to None.
            total (int, optional): Total(Max) number of results you want to get. If None, extracts all results. Defaults to None.
            pagination (bool, optional): Set to False if want to handle each page request manually. Use end_cursor from the previous page/request to navigate to the next page. Defaults to True.

        Returns:
            dict: Returns data, cursor_endpoint, has_next_page
        """
        user_id = self.get_user_id(user_id)
        query_endpoint = Path.USER_TWEETS_ENDPOINT
        variables = {"userId": user_id, "count": 100, "includePromotedContent": True,
                     "withQuickPromoteEligibilityTweetFields": True, "withVoice": True, "withV2Timeline": True}
        if with_replies:
            if not self.logged_in():
                self.login()
            variables["count"] = 20
            variables['withCommunity'] = True
            query_endpoint = Path.USER_TWEETS_AND_REPLIES_ENDPOINT
            del variables['withQuickPromoteEligibilityTweetFields']

        request_payload = self._generate_request_data(
            query_endpoint, variables, additional_features=True)
        data_path = ('data', 'user', 'result', 'timeline_v2',
                     'timeline', 'instructions')
        return self._handle_pagination(**request_payload, end_cursor=end_cursor, data_path=data_path, total=total, pagination=pagination)


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import asyncio
from crawl4ai import *

async def main():
    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(
            url="https://xcancel.com/omarsar0",
        )
        print(result.markdown)

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
export X_USERNAME='ai4fun2004'
export X_PASSWORD='ai4fun_test'
export X_EMAIL='ai4fun2004@gmail.com'


In [ ]:
import asyncio
from twikit import Client

USERNAME = 'ai4fun2004'
EMAIL = 'ai4fun2004@gmail.com'
PASSWORD = 'ai4fun_test'

# Initialize client
client = Client('en-US')

async def main():
    await client.login(
        auth_info_1=USERNAME,
        auth_info_2=EMAIL,
        password=PASSWORD,
        cookies_file='cookies.json'
    )

asyncio.run(main())

In [ ]:
user = await client.get_user_by_screen_name('omarsar0')

In [ ]:
user.id

In [ ]:
tweets = await client.get_user_tweets(user.id, 'Tweets')

tweets_info = []
for tweet in tweets:
    tweets_info.append(tweet)
    print(tweet.text)

In [ ]:
len(tweets_info)

In [ ]:
tweets_info[0]

In [ ]:
help(tweets_info[0])